## Getting the Sistema de Informações sobre Mortalidade (SIM) dataset

### Firstly we're going to access the microdata columns and their data dictionary to understand the available features and select the most interesting to our exploration. 

In [1]:
import basedosdados as bd

In [44]:
# get the headers

sim_header = bd.read_sql(query=""" SELECT * FROM basedosdados.br_ms_sim.microdados LIMIT 0 """,
                          billing_project_id='explorando-basedosdados')
sim_columns = sim_header.columns
sim_columns

Downloading: 0rows [00:00, ?rows/s]


Index(['ano', 'sigla_uf', 'sequencial_obito', 'tipo_obito', 'causa_basica',
       'data_obito', 'hora_obito', 'naturalidade', 'data_nascimento', 'idade',
       'sexo', 'raca_cor', 'estado_civil', 'escolaridade', 'ocupacao',
       'codigo_bairro_residencia', 'id_municipio_residencia',
       'local_ocorrencia', 'codigo_bairro_ocorrencia',
       'id_municipio_ocorrencia', 'idade_mae', 'escolaridade_mae',
       'ocupacao_mae', 'quantidade_filhos_vivos', 'quantidade_filhos_mortos',
       'gravidez', 'gestacao', 'parto', 'obito_parto', 'morte_parto', 'peso',
       'obito_gravidez', 'obito_puerperio', 'assistencia_medica', 'exame',
       'cirurgia', 'necropsia', 'linha_a', 'linha_b', 'linha_c', 'linha_d',
       'linha_ii', 'circunstancia_obito', 'acidente_trabalho', 'fonte',
       'codigo_estabelecimento', 'atestante', 'data_atestado', 'tipo_pos',
       'data_investigacao', 'causa_basica_original', 'data_cadastro',
       'fonte_investigacao', 'data_recebimento', 'causa_basica_pre

In [33]:
# get dictionary to inform the column selection

sim_dic = bd.read_sql(query=""" SELECT coluna, chave, valor
                                FROM basedosdados.br_ms_sim.dicionario
                                WHERE id_tabela = 'microdados'""",
                          billing_project_id='explorando-basedosdados')
sim_dic.to_csv('../data/sim_dic')

# check if the time cover is unique
# set(sim_dic.cobertura_temporal)
sim_dic

Downloading: 100%|██████████| 557/557 [00:02<00:00, 199.67rows/s]


,coluna,chave,valor
0,tipo_obito,1,Fetal
1,tipo_obito,2,Não-fetal
2,naturalidade,1,ABU DABI
3,naturalidade,2,ACORES
4,naturalidade,3,AFEGANISTAO
...,...,...,...
552,local_ocorrencia,1,Hospital
553,local_ocorrencia,2,Outro estabelecimento de saúde
554,local_ocorrencia,3,Domicílio
555,local_ocorrencia,4,Via pública


## Tools for explore the data

In [229]:
# consult a column in the data dictionary

def consult_sim_dic(term):
    #%timeit sim_dic.query("coluna == 'tipo_obito'")
    #%timeit sim_dic[ sim_dic['coluna'] == 'tipo_obito']
    # Boolean filter is faster than .query, and contains is more versatile, although slower
    #sim_dic[ sim_dic['coluna'] == 'local_ocorrencia']  

    return sim_dic[ sim_dic['coluna'].str.contains(term)]

consult_sim_dic('fonte')

,coluna,chave,valor
496,fonte,0,None
497,fonte,1,Boletim de ocorrência
498,fonte,2,Hospital
499,fonte,3,Família
500,fonte,4,Outro
501,fonte,5,None
502,fonte,6,None
503,fonte,7,None
504,fonte,9,None
512,fonte_investigacao,0,None


In [148]:
#Slicer to flick thougth the list of columns

def slicer(slice_index=0, delta=10):
    slice_index = 9
    delta = 10
    slice_ = slice(delta * slice_index, (slice_index*delta + delta))
    return slice_

sim_columns[slicer(9)].to_list()

['fontes_informacao', 'alt_causa']

## Using the tools to slice 10 coluns and investigate their values on the dictionary we selected the columns of interest

In [192]:
# Manually selected columns of interest.
# todo: investigate col to refine selection

sim_selected_col_loose = ('ano',
                          'sigla_uf',
                          'causa_basica',
                          'data_obito',
                          'hora_obito',
                          'data_nascimento',
                          'idade',
                          'sexo',
                          'raca_cor',
                          'estado_civil',
                          'escolaridade',
                          'ocupacao',
                          'id_municipio_residencia',
                          'local_ocorrencia',
                          'codigo_bairro_ocorrencia',
                          'id_municipio_ocorrencia',
                          'idade_mae',
                          'escolaridade_mae',
                          'ocupacao_mae',
                          'quantidade_filhos_vivos',
                          'quantidade_filhos_mortos',
                          'peso',
                          'assistencia_medica',
                          'circunstancia_obito',
                          'acidente_trabalho',
                          'fonte',
                          'data_atestado',
                          'causa_basica_original',
                          'causa_basica_pre',
                          'id_municipio_svo_iml',
                          'causa_materna',
                          'status_do_epidem',
                          'status_do_nova',
                          'serie_escolar_falecido',
                          'serie_escolar_mae',
                          'escolaridade_2010',
                          'escolaridade_mae_2010',
                          'escolaridade_falecido_2010_agr',
                          'escolaridade_mae_2010_agr',
                          'diferenca_data',
                          'id_municipio_naturalidade',
                          'descricao_estabelecimento',
                          'alt_causa')

sim_selected_col = ('ano',
                    'sigla_uf',
                    'causa_basica',
                    'data_obito',
                    'hora_obito',
                    'data_nascimento',
                    'idade',
                    'sexo',
                    'raca_cor',
                    'estado_civil',
                    'escolaridade',
                    'ocupacao',
                    'id_municipio_residencia',
                    'local_ocorrencia',
                    'id_municipio_ocorrencia',
                    'idade_mae',
                    'escolaridade_mae',
                    'ocupacao_mae',
                    'quantidade_filhos_vivos',
                    'quantidade_filhos_mortos',
                    'peso',
                    'assistencia_medica',
                    'circunstancia_obito',
                    'acidente_trabalho')


sim_col_investigate = set.difference(set(sim_selected_col_loose), set(sim_selected_col))
sim_col_investigate

{'alt_causa',
 'causa_basica_original',
 'causa_basica_pre',
 'causa_materna',
 'data_atestado',
 'descricao_estabelecimento',
 'diferenca_data',
 'escolaridade_2010',
 'escolaridade_falecido_2010_agr',
 'escolaridade_mae_2010',
 'escolaridade_mae_2010_agr',
 'fonte',
 'id_municipio_svo_iml',
 'serie_escolar_falecido',
 'serie_escolar_mae',
 'status_do_epidem',
 'status_do_nova'}

### Investigating columns

In [183]:
f"""{unpack(sim_col_investigate)}"""

'data_atestado, escolaridade_mae_2010_agr, fonte, id_municipio_svo_iml, serie_escolar_mae, causa_materna, causa_basica_pre, status_do_epidem, alt_causa, diferenca_data, serie_escolar_falecido, causa_basica_original, escolaridade_mae_2010, status_do_nova, escolaridade_falecido_2010_agr, escolaridade_2010, descricao_estabelecimento'

In [200]:
def unpack(l):
    return ", ".join(map(str, l)) 

query = f"""
            SELECT {unpack(sim_col_investigate)}
            FROM basedosdados.br_ms_sim.microdados
            WHERE rand() < 30/10000000
            LIMIT 30
        """

sim_investigate = bd.read_sql(query=query,
                              billing_project_id='explorando-basedosdados')
sim_investigate

# Reference
### Random sampling idea from: https://stackoverflow.com/a/14629551/15275978
### The random number is a guess over the row count 

Downloading: 100%|██████████| 30/30 [00:00<00:00, 104.80rows/s]


,data_atestado,escolaridade_mae_2010_agr,fonte,id_municipio_svo_iml,serie_escolar_mae,causa_materna,causa_basica_pre,status_do_epidem,alt_causa,diferenca_data,serie_escolar_falecido,causa_basica_original,escolaridade_mae_2010,status_do_nova,escolaridade_falecido_2010_agr,escolaridade_2010,descricao_estabelecimento
0,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
1,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
2,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
3,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
4,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
5,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
6,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
7,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
8,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None
9,NaT,None,None,None,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,None


We can clearly see that those columns hold little information

### Now lets explore the selected columns

In [203]:
query = f"""
            SELECT {unpack(sim_selected_col)}
            FROM basedosdados.br_ms_sim.microdados
            WHERE RAND() < 10/10000000
            LIMIT 10
        """

sim_sel = bd.read_sql(query=query,
                              billing_project_id='explorando-basedosdados')
sim_sel

Downloading: 100%|██████████| 10/10 [00:00<00:00, 24.50rows/s]


,ano,sigla_uf,causa_basica,data_obito,hora_obito,data_nascimento,idade,sexo,raca_cor,estado_civil,...,idade_mae,escolaridade_mae,ocupacao_mae,quantidade_filhos_vivos,quantidade_filhos_mortos,peso,assistencia_medica,circunstancia_obito,acidente_trabalho,id_municipio_naturalidade
0,1996,PE,I259,1996-12-25,NaT,1925-05-16,71.00,2,None,1,...,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
1,2015,RO,I629,2015-10-10,00:35:00,1939-07-18,76.00,1,2,3,...,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
2,2009,DF,C710,2009-12-31,14:30:00,1937-08-10,72.00,2,4,2,...,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None
3,2001,CE,Q909,2001-11-28,NaT,2001-10-12,0.08,1,1,None,...,<NA>,None,None,<NA>,<NA>,1500,None,None,None,None
4,1998,PE,J449,1998-06-30,NaT,1924-05-08,74.00,1,1,2,...,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
5,2007,MA,W860,2007-07-23,16:30:00,2004-09-09,2.00,2,4,None,...,<NA>,None,None,<NA>,<NA>,<NA>,None,1,None,None
6,2007,MA,J960,2007-07-13,02:00:00,1981-01-24,26.00,1,2,1,...,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None
7,1999,PE,J440,1999-10-12,NaT,1914-02-04,85.00,2,4,1,...,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
8,2012,MA,K566,2012-03-14,08:40:00,1944-10-01,67.00,2,4,2,...,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
9,2002,BA,R98,2002-12-28,NaT,1928-06-22,74.00,2,4,3,...,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None


In [226]:
sim_sel.iloc[:,10:]

,escolaridade,ocupacao,id_municipio_residencia,local_ocorrencia,codigo_bairro_ocorrencia,id_municipio_ocorrencia,idade_mae,escolaridade_mae,ocupacao_mae,quantidade_filhos_vivos,quantidade_filhos_mortos,peso,assistencia_medica,circunstancia_obito,acidente_trabalho,id_municipio_naturalidade
0,None,00700,2611606,1,None,2611606,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
1,2,622020,1100205,1,None,1100205,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
2,None,999992,5300108,3,00000048,5300108,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None
3,None,None,2301802,1,None,2313708,<NA>,None,None,<NA>,<NA>,1500,None,None,None,None
4,None,00700,2611606,1,None,2611606,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
5,None,None,2110104,3,00000099,2110104,<NA>,None,None,<NA>,<NA>,<NA>,None,1,None,None
6,3,261715,2103307,1,None,2211001,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None
7,4,00700,2609600,3,None,2609600,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
8,2,621005,2114007,1,None,2211001,<NA>,None,None,<NA>,<NA>,<NA>,1,None,None,None
9,None,00700,2924009,3,None,2924009,<NA>,None,None,<NA>,<NA>,<NA>,None,None,None,None


In [235]:
consult_sim_dic('local')

,coluna,chave,valor
552,local_ocorrencia,1,Hospital
553,local_ocorrencia,2,Outro estabelecimento de saúde
554,local_ocorrencia,3,Domicílio
555,local_ocorrencia,4,Via pública
556,local_ocorrencia,5,Outros


In [234]:
sim_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ano                        10 non-null     Int64  
 1   sigla_uf                   10 non-null     object 
 2   causa_basica               10 non-null     object 
 3   data_obito                 10 non-null     dbdate 
 4   hora_obito                 5 non-null      dbtime 
 5   data_nascimento            10 non-null     dbdate 
 6   idade                      10 non-null     float64
 7   sexo                       10 non-null     object 
 8   raca_cor                   9 non-null      object 
 9   estado_civil               8 non-null      object 
 10  escolaridade               4 non-null      object 
 11  ocupacao                   8 non-null      object 
 12  id_municipio_residencia    10 non-null     object 
 13  local_ocorrencia           10 non-null     object 
 1